# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 8:34AM,244122,20,ALUR376824361,Alumier Labs Inc.,Released
1,Aug 15 2022 8:34AM,244122,20,ALUR440639669,Alumier Labs Inc.,Released
2,Aug 15 2022 8:34AM,244122,20,ALUR488561504,Alumier Labs Inc.,Released
3,Aug 15 2022 8:34AM,244122,20,ALUR567555815,Alumier Labs Inc.,Released
4,Aug 15 2022 8:34AM,244122,20,ALUR657136146,Alumier Labs Inc.,Released
5,Aug 15 2022 8:34AM,244122,20,ALU0007546,Alumier Labs Inc.,Released
6,Aug 15 2022 8:32AM,244126,19,60017847,"GUSA Granules USA, Inc.",Released
7,Aug 15 2022 8:30AM,244107,10,MSP213078,"Methapharm, Inc.",Released
8,Aug 15 2022 8:30AM,244107,10,MSP213079,"Methapharm, Inc.",Released
9,Aug 15 2022 8:30AM,244107,10,MSP213080,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,244120,Released,2
9,244121,Released,3
10,244122,Released,6
11,244123,Released,8
12,244126,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
244120,2
244121,3
244122,6
244123,8
244126,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
244107,10
244108,11
244109,27
244110,15
244111,1


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
244107,10
244108,11
244109,27
244110,15
244111,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,244107,10
1,244108,11
2,244109,27
3,244110,15
4,244111,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,244107,10
1,244108,11
2,244109,27
3,244110,15
4,244111,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.
6,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc."
7,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc."
17,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation
28,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation
55,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation
70,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health
71,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC"
79,Aug 15 2022 8:12AM,244121,10,Emerginnova
82,Aug 15 2022 8:08AM,244120,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6
1,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1
2,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10
3,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11
4,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27
5,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15
6,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1
7,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC",8
8,Aug 15 2022 8:12AM,244121,10,Emerginnova,3
9,Aug 15 2022 8:08AM,244120,20,"Exact-Rx, Inc.",2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6
1,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1
2,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10
3,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11
4,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27
5,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15
6,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1
7,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC",8
8,Aug 15 2022 8:12AM,244121,10,Emerginnova,3
9,Aug 15 2022 8:08AM,244120,20,"Exact-Rx, Inc.",2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6
1,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1
2,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10
3,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11
4,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6
1,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1
2,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10
3,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11
4,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6
1,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1
2,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10
3,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11
4,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,1708789,75
12,488237,36
19,488242,29
20,488242,8


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,1708789,75
1,12,488237,36
2,19,488242,29
3,20,488242,8


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,75
1,12,36
2,19,29
3,20,8


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,75
1,12,Released,36
2,19,Released,29
3,20,Released,8


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Released,75,36,29,8


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Released,75,36,29,8


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Released,75,36,29,8


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()